### Part One

We start by defining a `Registers` class. This is essentially a dictionary with methods that describe our domain in order to make the code later more readable.

In [1]:
from collections import defaultdict


def value(d, x):
    try:
        return int(x)
    except ValueError:
        return int(d[x])
    

class Registers(defaultdict):
    def set(self, x, y):
        self[x] = value(self, y)
        
    def multiply(self, x, y):
        self[x] *= value(self, y)
        
    def jump(self, x, y):
        x = value(self, x)
        if x > 0:
            return value(self, y)
    
    def add(self, x, y):
        self[x] += value(self, y)
    
    def mod(self, x, y):
        self[x] = value(self, x) % value(self, y)
    
    def play(self, x):
        self['last_value_played'] = value(self, x)
    
    def recover(self, x):
        if value(self, x) > 0:
            return str(self['last_value_played'])

Let's also define a `parse` function that can read the instructions provided.

In [2]:
map_ = {
    'set': Registers.set,
    'mul': Registers.multiply,
    'jgz': Registers.jump,
    'add': Registers.add,
    'mod': Registers.mod,
    'snd': Registers.play,
    'rcv': Registers.recover,
}


def parse(string, map_=map_):
    f_string, *args_string = string.split(' ')
    try:
        return map_[f_string], args_string
    except KeyError:
        raise ValueError(f'Unable to parse: {string}')

Now let's build a controller that can execute the instructions and coordinate the output.

In [3]:
def read_example():
    return [
        'set a 1',
        'add a 2',
        'mul a a',
        'mod a 5',
        'snd a',
        'set a 0',
        'rcv a',
        'jgz a -1',
        'set a 1',
        'jgz a -2',
    ]


def controller(instructions):
    registers = Registers(lambda: 0)
    position = 0
    
    while True:
        f, args = instructions[position]
        response = f(registers, *args)
    
        if response is None:
            position += 1
        elif isinstance(response, int):
            position += response
        elif isinstance(response, str):
            return response
        

instructions = list(map(parse, read_example()))
controller(instructions)

'4'

Finally for the puzzle we have

In [4]:
def read():
    with open('../../data/day18.txt') as f:
        data = f.read().strip()
        
    return data.split('\n')


instructions = list(map(parse, read()))
controller(instructions)

'9423'

### Test cases

First, let's test that instructions can be parsed correctly.

In [5]:
assert parse('set i 31') == (Registers.set, ['i', '31'])
assert parse('mul p 17') == (Registers.multiply, ['p', '17'])
assert parse('jgz -2 i') == (Registers.jump, ['-2', 'i'])
assert parse('add i -1') == (Registers.add, ['i', '-1'])
assert parse('mod 3 a') == (Registers.mod, ['3', 'a'])
assert parse('snd a') == (Registers.play, ['a'])
assert parse('rcv b') == (Registers.recover, ['b'])

Then let's test that the register operations do what we expect.

In [6]:
test_cases = [
    (Registers.set, ['a', '3'], {'a': 3, 'b': 3}, None),
    (Registers.set, ['a', 'b'], {'a': 3, 'b': 3}, None),
    (Registers.multiply, ['a', '3'], {'a': 6, 'b': 3}, None),
    (Registers.multiply, ['a', 'b'], {'a': 6, 'b': 3}, None),
    (Registers.jump, ['0', '2'], {'a': 2, 'b': 3}, None),
    (Registers.jump, ['1', '2'], {'a': 2, 'b': 3}, 2),
    (Registers.jump, ['1', 'a'], {'a': 2, 'b': 3}, 2),
    (Registers.add, ['a', '3'], {'a': 5, 'b': 3}, None),
    (Registers.add, ['a', 'b'], {'a': 5, 'b': 3}, None),
    (Registers.mod, ['b', '2'], {'a': 2, 'b': 1}, None),
    (Registers.mod, ['b', 'a'], {'a': 2, 'b': 1}, None),
]

for function, args, expected_registers, expected_return in test_cases:
    registers = {'a': 2, 'b': 3}
    assert expected_return == function(registers, *args), function(registers, *args)
    assert expected_registers == registers, f'Expected: {expected_registers}, received: {registers}'

Next let's look at the register `play` and `recover` functions.

In [7]:
registers = defaultdict(lambda: 0)
registers['a'] = 2
registers['b'] = 3

assert None == Registers.play(registers, 'a')
assert None == Registers.recover(registers, '0')
assert '2' == Registers.recover(registers, 'b')

assert None == Registers.play(registers, '4')
assert '4' == Registers.recover(registers, 'b')